**Graph RAG (Retrieval-Augmented Generation) Framework for the Social Network Example Using Graph Databases**  

### **Graph RAG Description**  
Graph RAG integrates graph databases (like Neo4j) into Retrieval-Augmented Generation systems, leveraging their ability to model relationships in data for enhanced query understanding, reasoning, and content generation. The system retrieves data from the graph to provide context for generation tasks, enhancing the accuracy and relevance of the responses.

---

### **Graph RAG Components**  

1. **Knowledge Base (Graph Database):**  
   - **Structure:** Nodes, relationships, and properties.  
   - **Data Model:** The example social network modeled with users, posts, and interactions.

2. **Retrieval Mechanism:**  
   - **Query Language:** Cypher queries to extract relationships, content, and user interactions.  
   - **Context Assembly:** Retrieved graph data serves as the context for generation.

3. **Generation Layer:**  
   - **LLM (Language Model):** Generates natural language responses based on retrieved graph data.  
   - **Templates:** Structures for assembling responses (e.g., summarizing social interactions or providing recommendations).

---

### **Graph Structure in Neo4j**

#### **Nodes and Relationships**  
- **Nodes:**  
  - `User`: `{name, age}`  
  - `Post`: `{content, timestamp}`  

- **Relationships:**  
  - `FRIEND`: Friendship between users.  
  - `POSTED`: Users posting content.  
  - `LIKES`: Users liking posts.  

#### **Graph Representation**  
```cypher
// Create Users
CREATE (alice:User {name: 'Alice', age: 30}),
       (bob:User {name: 'Bob', age: 25}),
       (carol:User {name: 'Carol', age: 27});

// Create Posts
CREATE (post1:Post {content: 'Graph databases are cool!', timestamp: '2024-05-20'}),
       (post2:Post {content: 'Learning Cypher is fun!', timestamp: '2024-05-19'});

// Define Relationships
CREATE (alice)-[:FRIEND]->(bob),
       (bob)-[:FRIEND]->(carol),
       (carol)-[:FRIEND]->(alice),
       (alice)-[:POSTED]->(post1),
       (bob)-[:POSTED]->(post2),
       (bob)-[:LIKES]->(post1),
       (carol)-[:LIKES]->(post1),
       (alice)-[:LIKES]->(post2);
```

---

### **Retrieval Queries (Cypher)**  

1. **Find Alice's Friends:**  
```cypher
MATCH (alice:User {name: 'Alice'})-[:FRIEND]->(friends)
RETURN friends.name;
```

2. **Posts Liked by Alice:**  
```cypher
MATCH (alice:User {name: 'Alice'})-[:LIKES]->(posts)
RETURN posts.content;
```

3. **Who Liked a Specific Post:**  
```cypher
MATCH (post:Post {content: 'Graph databases are cool!'})<-[:LIKES]-(users)
RETURN users.name;
```

---

### **Integration with the Generation Layer**  

**Prompt Assembly for RAG Context:**  
The retrieved data is used to form the prompt for the LLM.  
Example Prompt:  
> "Alice has 2 friends: Bob and Carol. She liked the post 'Learning Cypher is fun!'. Who are the users that liked the post 'Graph databases are cool!'?"

**Generated Response:**  
> "The users who liked the post 'Graph databases are cool!' are Bob and Carol."

---

### **Use Cases of Graph RAG in Social Networks**

1. **Recommendation Systems:**  
   - Retrieve connected entities (e.g., "Friends of friends who liked similar posts").  

2. **Insight Generation:**  
   - Analyze and summarize user behavior for insights, e.g., "Alice and her friends prefer technical content like Cypher."

3. **Fraud Detection:**  
   - Traverse relationships to detect anomalies, e.g., suspicious clusters of 'likes' or repeated patterns.

---

### **Benefits of Graph RAG**  

- **Efficiency:** Graph databases enhance retrieval speed and handle complex relationships.  
- **Contextual Accuracy:** Leveraging graph structure ensures highly relevant contextual data for the LLM.  
- **Scalability:** Suitable for interconnected and growing datasets like social networks.  

This notebook highlights the potential of Graph RAG to integrate Neo4j's graph database with generative AI for advanced, relationship-driven query processing and response generation.

In [1]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.2 MB/s eta 0:00:00


In [2]:
## Graphdb configuration
NEO4J_URI="neo4j+s://2712b7a4.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="VHhDstBATnsK6xsfJjdWl5Q8Mvhzc1GkJPuFLeEObwM"

In [3]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [4]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [5]:
graph

In [6]:
groq_api_key="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"

In [7]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fee0eafae30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fee0eaf8250>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.documents import Document
text="""
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics
 and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
  two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
 """
documents=[Document(page_content=text)]
documents

[Document(metadata={}, page_content="\nElon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space\ncompany SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,\nformerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.\nHe is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be\nUS$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended\nthe University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through\nhis Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.\nMusk later transferred to the University of Pennsylvania and received bachelor's degrees in economics\n and physics. He moved to California in 1995 to attend Stanford University, but dropped out after\n  two days and, with his brother Kimbal, co-founded onlin

In [9]:
!pip install --upgrade --quiet langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 5.0 MB/s eta 0:00:00


In [10]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [11]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [40]:
# graph_documents

In [13]:
graph_documents[0].nodes

[Node(id='Elon Reeve Musk', type='Person', properties={}),
 Node(id='Maye', type='Person', properties={}),
 Node(id='Errol Musk', type='Person', properties={}),
 Node(id='Kimbal Musk', type='Person', properties={}),
 Node(id='Spacex', type='Company', properties={}),
 Node(id='Tesla, Inc.', type='Company', properties={}),
 Node(id='X Corp.', type='Company', properties={}),
 Node(id='Twitter', type='Company', properties={}),
 Node(id='The Boring Company', type='Company', properties={}),
 Node(id='Xai', type='Company', properties={}),
 Node(id='Neuralink', type='Company', properties={}),
 Node(id='Openai', type='Company', properties={}),
 Node(id='University Of Pretoria', type='University', properties={}),
 Node(id="Queen'S University", type='University', properties={}),
 Node(id='University Of Pennsylvania', type='University', properties={}),
 Node(id='Stanford University', type='University', properties={}),
 Node(id='Zip2', type='Company', properties={})]

In [14]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Maye', type='Person', properties={}), type='PARENT', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Errol Musk', type='Person', properties={}), type='PARENT', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Kimbal Musk', type='Person', properties={}), type='SIBLING', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Spacex', type='Company', properties={}), type='FOUNDER', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Tesla, Inc.', type='Company', properties={}), type='FOUNDER', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='X Corp.', type='Company', properties={}), type='OWNER', pr

In [15]:
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [16]:
graph

In [17]:
graph.query(movie_query)

[]

In [18]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING, id: STRING}
Genre {name: STRING}
Document {id: STRING, title: STRING, text: STRING, summary: STRING, source: STRING, embedding: LIST}
Country {id: STRING}
Place {id: STRING}
Building {id: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:PARENT]->(:Person)
(:Person)-[:SIBLING]->(:Person)
(:Person)-[:RELATIVE]->(:Person)
(:Person)-[:ADVISER]->(:Person)
(:Person)-[:MINISTER]->(:Person)
(:Person)-[:CHILD_OF]->(:Person)
(:Person)-[:GRANDMOTHER]->(:Person)
(:Person)-[:AT_WAR_WITH]->(:Country)
(:Person)-[:MILITARY_CAMPAIGN]->(:Country)
(:Person)-[:RULER_DURING]->(:Person)
(:Person)-[:CHILD]->(:Person)
(:Person)-[:SPOUSE]->(:Person)
(:Person)-[:DAUGHTER]->(:Person)
(:Person)-[:HEIR]->(:Person)
(:Person)-[:SUPPORTED]->(:Person)
(:Person)-[:COLLABORATED_WITH]->(:Person)
(:Person)-

In [19]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    llm=llm, graph=graph, verbose=True, allow_dangerous_requests=True
)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7fee4150d930>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fee0eafae30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fee0eaf8250>, model_name='Gemma2-9b-I

In [20]:
response=chain.invoke({"query":"Who was the director of the moview GoldenEye"})

response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'GoldenEye'})-[:DIRECTED]->(p:Person) RETURN p.name
Full Context:
[]

> Finished chain.


{'query': 'Who was the director of the moview GoldenEye',
 'result': "I don't know the answer. \n"}

In [21]:
response=chain.invoke({"query":"tell me the genre of th movie GoldenEye"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})<-[:IN_GENRE]-(g:Genre) RETURN g.name  

Full Context:
[]

> Finished chain.


{'query': 'tell me the genre of th movie GoldenEye',
 'result': "I don't know the answer. \n"}

In [22]:
response=chain.invoke({"query":"Who was the director in movie Casino"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Casino'})-[:DIRECTED]->(p:Person) RETURN p.name
Full Context:
[]

> Finished chain.


{'query': 'Who was the director in movie Casino',
 'result': "I don't know the answer. \n"}

In [23]:

response=chain.invoke({"query":"Which movie were released in 2008"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)
WHERE m.released = "2008"
RETURN m.title
Full Context:
[]

> Finished chain.


{'query': 'Which movie were released in 2008',
 'result': "I don't know the answer. \n"}

In [24]:
response=chain.invoke({"query":"Give me the list of movie having imdb rating more than 8"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.imdbRating > 8 RETURN m.title
Full Context:
[{'m.title': 'Toy Story'}, {'m.title': 'Heat'}, {'m.title': 'Casino'}, {'m.title': 'Twelve Monkeys (a.k.a. 12 Monkeys)'}, {'m.title': 'Seven (a.k.a. Se7en)'}, {'m.title': 'Usual Suspects, The'}, {'m.title': 'Hate (Haine, La)'}, {'m.title': 'Braveheart'}, {'m.title': 'Taxi Driver'}, {'m.title': 'Anne Frank Remembered'}]

> Finished chain.


{'query': 'Give me the list of movie having imdb rating more than 8',
 'result': "I don't know the answer. \n"}

In [25]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [26]:
%pip install --upgrade --quiet  langchain langchain-community langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.8 MB/s eta 0:00:00


In [27]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)


from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

from typing import Tuple, List, Optional


from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser


from langchain_core.runnables import ConfigurableField


from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector




import os


try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [28]:
graph = Neo4jGraph()
graph

In [29]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Elizabeth I").load()
print(len(raw_documents))
# raw_documents

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


23


In [47]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents_1 = text_splitter.split_documents(raw_documents[:3])
# documents

In [48]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)
# llm_transformer

In [49]:
graph_documents_1 = llm_transformer.convert_to_graph_documents(documents_1)
# graph_documents

In [50]:
graph.add_graph_documents(
    graph_documents_1,
    baseEntityLabel=True,
    include_source=True
)


# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [51]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass


def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [52]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [37]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )



from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [41]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Elizabeth I").load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [54]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)


graph_doc = llm_transformer.convert_to_graph_documents(documents)


# graph_documents


In [55]:
graph.add_graph_documents(
    graph_doc,
    baseEntityLabel=True,
    include_source=True
)


# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"


from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass


def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget


showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))